# Interacting with CLIP

이번 실습에서는 CLIP 모델을 다운로드하고 실행하며, 이미지와 텍스트 입력 간의 유사성을 계산하여, zero-shot 이미지 분류를 수행하는 방법을 실행해봅니다.

# Preparation for Colab

GPU 런타임을 실행 중인지 확인하세요. 그렇지 않다면 메뉴에서 Runtime > Change Runtime Type으로 가서 하드웨어 가속기를 "GPU"로 선택하세요. 다음 셀은 clip 패키지와 그 종속 항목들을 설치하고, PyTorch 1.7.1 이상 버전이 설치되어 있는지 확인할 것입니다.

In [ ]:
! pip install ftfy regex tqdm
! pip install git+https://github.com/openai/CLIP.git

In [ ]:
import numpy as np
import torch
from pkg_resources import packaging

print("Torch version:", torch.__version__)

# Loading the model

`clip.available_models()`는 사용 가능한 CLIP 모델의 이름을 나열합니다.

In [ ]:
import clip

clip.available_models()

In [ ]:
model, preprocess = clip.load("ViT-B/32")
model.cuda().eval()
input_resolution = model.visual.input_resolution
context_length = model.context_length
vocab_size = model.vocab_size

print("Model parameters:", f"{np.sum([int(np.prod(p.shape)) for p in model.parameters()]):,}")
print("Input resolution:", input_resolution)
print("Context length:", context_length)
print("Vocab size:", vocab_size)

# Image Preprocessing

모델이 예상하는 이미지로 전처리하기 위해, 데이터셋의 평균과 표준 편차를 사용하여 픽셀 강도를 정규화합니다. 이후 모델 입력 해상도에 맞추기 위해 크기를 조정하고 center-crop합니다.

clip.load()의 반환 값에는 이러한 전처리를 수행하는 torchvision의 Transform을 포함합니다.



In [ ]:
preprocess

# Text Preprocessing

우리는 대소문자를 구분하지 않는 토크나이저를 사용하며, 이는 `clip.tokenize()`를 사용하여 호출할 수 있습니다. 기본적으로 출력은 CLIP 모델이 기대하는 77개 토큰 길이로 패딩됩니다.

In [ ]:
clip.tokenize("Hello World!")

# Setting up input images and texts

8개의 예제 이미지와 그에 대한 텍스트 설명을 모델에 입력하고, 해당하는 특징들 간의 유사성을 비교할 것입니다.

토크나이저는 대소문자를 구분하지 않으며, 우리는 자유롭게 텍스트 설명을 제공할 수 있습니다.

In [ ]:
import os
import skimage
import IPython.display
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

from collections import OrderedDict
import torch

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# images in skimage to use and their textual descriptions
descriptions = {
    "page": "a page of text about segmentation",
    "chelsea": "a facial photo of a tabby cat",
    "astronaut": "a portrait of an astronaut with the American flag",
    "rocket": "a rocket standing on a launchpad",
    "motorcycle_right": "a red motorcycle standing in a garage",
    "camera": "a person looking at a camera on a tripod",
    "horse": "a black-and-white silhouette of a horse",
    "coffee": "a cup of coffee on a saucer"
}

In [ ]:
original_images = []
images = []
texts = []
plt.figure(figsize=(16, 5))

data_dir = "/usr/local/lib/python3.10/dist-packages/skimage/data"

for filename in [filename for filename in os.listdir(data_dir) if filename.endswith(".png") or filename.endswith(".jpg")]:
    name = os.path.splitext(filename)[0]
    if name not in descriptions:
        continue

    image = Image.open(os.path.join(data_dir, filename)).convert("RGB")

    plt.subplot(2, 4, len(images) + 1)
    plt.imshow(image)
    plt.title(f"{filename}\n{descriptions[name]}")
    plt.xticks([])
    plt.yticks([])

    original_images.append(image)
    images.append(preprocess(image))
    texts.append(descriptions[name])

plt.tight_layout()


## Building features

우리는 이미지를 정규화하고, 각 텍스트 입력을 토큰화한 다음, 모델의 순방향 패스를 실행하여 이미지와 텍스트 특징을 얻습니다.

In [ ]:
image_input = torch.tensor(np.stack(images)).cuda()
text_tokens = clip.tokenize(["This is " + desc for desc in texts]).cuda()

In [ ]:
with torch.no_grad():
    image_features = model.encode_image(image_input).float()
    text_features = model.encode_text(text_tokens).float()

## Calculating cosine similarity

우리는 특징을 정규화하고 각 쌍의 내적을 계산합니다.

In [ ]:
image_features /= image_features.norm(dim=-1, keepdim=True)
text_features /= text_features.norm(dim=-1, keepdim=True)
similarity = text_features.cpu().numpy() @ image_features.cpu().numpy().T

In [ ]:
count = len(descriptions)

plt.figure(figsize=(20, 14))
plt.imshow(similarity, vmin=0.1, vmax=0.3)
# plt.colorbar()
plt.yticks(range(count), texts, fontsize=18)
plt.xticks([])
for i, image in enumerate(original_images):
    plt.imshow(image, extent=(i - 0.5, i + 0.5, -1.6, -0.6), origin="lower")
for x in range(similarity.shape[1]):
    for y in range(similarity.shape[0]):
        plt.text(x, y, f"{similarity[y, x]:.2f}", ha="center", va="center", size=12)

for side in ["left", "top", "right", "bottom"]:
  plt.gca().spines[side].set_visible(False)

plt.xlim([-0.5, count - 0.5])
plt.ylim([count + 0.5, -2])

plt.title("Cosine similarity between text and image features", size=20)

# 실습과제 Zero-Shot Image Classification 구현하기

이 과제는 CLIP 모델을 활용하여 CIFAR-100 이미지 데이터셋에 대한 zero-shot classification을 구현하는 과제입니다. Zero-shot classification은 모델이 학습 과정에서 본 적 없는 새로운 클래스를 분류할 수 있는 능력을 의미합니다. CLIP은 이미지와 텍스트 사이의 일반적인 시맨틱 정보를 학습하여, 주어진 텍스트 설명과 가장 잘 맞는 이미지를 찾아내는 데 사용됩니다.

텍스트 설명 준비: CIFAR-100의 각 클래스에 해당하는 텍스트 설명을 준비합니다. 예를 들어, 클래스 이름이 "apple"이면, 해당 텍스트 설명은 "This is a photo of a apple"이 됩니다. 이런 식으로 모든 클래스에 대한 설명을 생성합니다.

In [ ]:
from torchvision.datasets import CIFAR100

cifar100 = CIFAR100(os.path.expanduser("~/.cache"), transform=preprocess, download=True)

In [ ]:
text_descriptions = [f"This is a photo of a {label}" for label in cifar100.classes]
text_tokens = clip.tokenize(text_descriptions).cuda()

이제 다음 단계를 직접 구현할 차례입니다:

텍스트 특성 추출: 토큰화된 텍스트를 CLIP 모델을 통해 인코딩하고, 각 텍스트 설명의 특성을 추출합니다.

이미지 특성 추출: CIFAR-100 데이터셋의 이미지를 전처리하고, CLIP 모델을 사용하여 이미지에서 특성을 추출합니다.

이미지-텍스트 매칭: 이미지 특성과 텍스트 특성 간의 유사도를 계산하여, 각 이미지에 가장 잘 맞는 텍스트 레이블(클래스)을 찾습니다.

결과 출력: 사진에 대해 예측된 클래스와 그 확률을 출력합니다.(가능하다면 top 5 까지의 class의 확률을 출력합니다.)

In [ ]:
#코드를 작성하시오.












